--------------
**Duplicate Removal in Original Datasets**

The purpose of this notebook is to create a unique tables that contain de-duplicated aggregate data per customer and timestep.

This notebook removes duplicates from 2 tables in the actor_data: wallet_features and wallet_features_classes_combined. We do not need duplicates in the aggregate data tables because duplicates can result in misleading analysis.

Elliptic1 Paper and Dataset:
https://arxiv.org/pdf/1908.02591
https://www.kaggle.com/datasets/ellipticco/elliptic-data-set/data

Elliptic++ Paper and Dataset:
https://arxiv.org/pdf/2306.06108
https://github.com/git-disl/EllipticPlusPlus


--------------


In [ ]:
# Data cleaning and manipulation
import pandas as pd
import numpy as np
from pandas_gbq import to_gbq

# Set up display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)


----------------------

Removing duplicates in 'df_wallet_combined' dataset.

----------------------

In [ ]:
# Read in combined wallet features table from BigQuery
%%bigquery df_wallet_combined
select * from `sixth-legend-440110-g7.actor_data.wallets_features_classes_combined`;


Query is running:   0%|          |

Downloading:   0%|          |

The below dataset shows the aggregated transactions sent, received with values across multiple timestamps. However, this dataset contains duplicates because the aggregated values per address are duplicated due the way the dataset was joined, which has resulted in the address information being multipled by the number of transactions the address has made across all timestamps.

We can see that there are 1.2m rows across 58 columns. However, there are only 823k unique addresses. This means that there are many duplicate rows we need to remove.

In [ ]:
df_wallet_combined.head(3)

,address,Time step,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,12m825yQ4vhS2zrd7ynZT4bBqKSMZYxuku,34,3,1.0000,0.0000,457729.0000,457729.0000,0.0000,1.0000,457729.0000,0.0000,1.0000,0.0427,0.0427,0.0427,0.0427,0.0427,0.0427,0.0427,0.0427,0.0427,0.0427,0.0000,0.0000,0.0000,0.0000,0.0000,0.0028,0.0028,0.0028,0.0028,0.0028,0.0003,0.0003,0.0003,0.0003,0.0003,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,1.0000,1.0000,1.0000,1.0000
1,1PUD8hRHaj9B8xPrbmTSFFWzuyG7wkP4xg,1,3,1.0000,0.0000,391201.0000,391201.0000,0.0000,1.0000,391201.0000,0.0000,1.0000,0.0048,0.0048,0.0048,0.0048,0.0048,0.0048,0.0048,0.0048,0.0048,0.0048,0.0000,0.0000,0.0000,0.0000,0.0000,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,1.0000,1.0000,1.0000,1.0000
2,1WMbzTvEfKfCiXNPLQJzSWS3ywqRMNgd5,2,2,1.0000,0.0000,393223.0000,393223.0000,0.0000,1.0000,393223.0000,0.0000,1.0000,0.0154,0.0154,0.0154,0.0154,0.0154,0.0154,0.0154,0.0154,0.0154,0.0154,0.0000,0.0000,0.0000,0.0000,0.0000,0.0057,0.0057,0.0057,0.0057,0.0057,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,1.0000,1.0000,1.0000,1.0000


In [ ]:
df_wallet_combined.shape

(1268260, 58)

In [ ]:
df_wallet_combined['address'].drop_duplicates().count()

822942

The example below shows the 'df_wallet_combined' dataset which has duplicate values. The duplication is because the aggregate data per address is multiplied by the total number of transactions the address has across all timesteps. The duplication is the result of the way the data was joined by the original owners.

In this example, '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b' has 26 total transactions across 14 timesteps, as seen in the two fields 'total_txs' and 'num_timesteps_appeared_in'. We can see that 12 transactions were sent by the address to other addresses and 14 were received by the address from other addresses.

In order to remove duplicates in 'df_wallet_combined', we need to remove the 'time step' field and then remove the duplicates.



In [ ]:
df_wallet_combined[df_wallet_combined['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,Time step,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
45692,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,48,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
108373,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,29,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
143658,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,24,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
175157,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,47,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
208423,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,5,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
208424,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,44,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.007

Remove the 'timestep' field and then remove duplicates. The expected result is a de-duplicated dataset with 822,942 representing 1 row per address.

In [ ]:
df_wallet_combined_clean = df_wallet_combined.drop(columns=['Time step'])
df_wallet_combined_clean = df_wallet_combined_clean.drop_duplicates()
df_wallet_combined_clean.shape

(822942, 57)

Now we can see that the 'Time step' field and the duplicates have been removed. The outcome is 1 row per address, resulting in 822,942 rows and 57 columns.

In [ ]:
df_wallet_combined_clean[df_wallet_combined_clean['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
45692,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,2,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000


Next, I am going to map the class label to the class number. This will allow me to determine the class (illicit, licit or unknown) without manually looking it up.

In [ ]:
# Map classes to a name
df_wallet_combined_clean['class_label'] = df_wallet_combined_clean['class'].map({1: 'Illicit', 2: 'Licit', 3: 'Unknown'})

In [ ]:
# Move new class label column to position 3, next to 'class' field.
column_to_move = 'class_label'  # The column at the end
target_position = 2       # 0-based index for the third position

col_data = df_wallet_combined_clean.pop(column_to_move)  # Remove the column
df_wallet_combined_clean.insert(target_position, column_to_move, col_data)  # Insert it at the target position
df_wallet_combined_clean.shape

(822942, 58)

In [ ]:
df_wallet_combined_clean[df_wallet_combined_clean['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,class,class_label,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
45692,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,2,Licit,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000


The final step in cleaning the 'df_wallet_combined' dataset is to save it as a separate table in BigQuery.

First, we define which project and the ID we want to save it as.

In [ ]:
# Define your project ID and table ID
project_id = 'sixth-legend-440110-g7'
table1_id = 'actor_data.wallets_combined_clean'

Then we save the cleaned df as a new table.

In [ ]:
# Save DataFrame to BigQuery
to_gbq(df_wallet_combined_clean, table1_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6384.02it/s]


Check that the new table we created reflects what we uploaded.

In [ ]:
# Read in combined wallet features table from BigQuery
%%bigquery df_wallet_combined_clean
select * from `sixth-legend-440110-g7.actor_data.wallets_combined_clean`;


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_wallet_combined_clean.shape

(822942, 58)

In [ ]:
df_wallet_combined_clean[df_wallet_combined_clean['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,class,class_label,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
596647,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,2,Licit,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000


----------------------

Removing duplicates in 'df_wallets_features' dataset.

----------------------

This dataset is similar to the 'df_wallet_combined' dataset in that it has the aggregated transaction values per address but does not include the class per address.

This dataset also contains duplicates in the original dataset which multiply the number of rows per address by the number of transactions the address made (both sending and receiving) across all timesteps.

The goal is to remove the timestep and duplicates in the aggregate data, using the same steps as we did in 'df_wallet_combined'.


In [ ]:
# Read in wallet features table from BigQuery
%%bigquery df_wallets_features
select * from `sixth-legend-440110-g7.actor_data.wallets_features`;


Query is running:   0%|          |

Downloading:   0%|          |

The below dataset shows the aggregated transactions sent, received with values across multiple timestamps but unlike 'df_wallet_combined', it does not contain the class of each address. this dataset contains duplicates because the aggregated values per address are duplicated due the way the original dataset was joined, which has resulted in the address information being multipled by the number of transactions the address has made across all timestamps.

We can see that there are 1.2m rows across 57 columns. However, there are only 823k unique addresses. This means that there are many duplicate rows we need to remove.

In [ ]:
df_wallets_features.head(3)

,address,Time step,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,184HTiqyG4Lb8fStjWZuVTGmr3ZTbpsmxH,30,1.0000,0.0000,449668.0000,449668.0000,0.0000,1.0000,449668.0000,0.0000,1.0000,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0000,0.0000,0.0000,0.0000,0.0000,0.0004,0.0004,0.0004,0.0004,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.0000,1.0000,1.0000,1.0000,1.0000
1,18TXPH2FFieGzVktuEG5rnWLdVRJH3TEjZ,10,1.0000,0.0000,409349.0000,409349.0000,0.0000,1.0000,409349.0000,0.0000,1.0000,0.0232,0.0232,0.0232,0.0232,0.0232,0.0232,0.0232,0.0232,0.0232,0.0232,0.0000,0.0000,0.0000,0.0000,0.0000,0.0001,0.0001,0.0001,0.0001,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,1.0000,1.0000,1.0000,1.0000
2,1PaqwoNxUDnH3Q4dHMw5WEE3vBNVaX9oGr,25,1.0000,0.0000,439589.0000,439589.0000,0.0000,1.0000,439589.0000,0.0000,1.0000,0.0101,0.0101,0.0101,0.0101,0.0101,0.0101,0.0101,0.0101,0.0101,0.0101,0.0000,0.0000,0.0000,0.0000,0.0000,0.0004,0.0004,0.0004,0.0004,0.0004,0.0004,0.0004,0.0004,0.0004,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,1.0000,1.0000,1.0000,1.0000


In [ ]:
df_wallets_features.shape

(1268260, 57)

In [ ]:
df_wallets_features['address'].drop_duplicates().count()

822942

The example below shows the 'df_wallets_features' dataset which has duplicate values. The duplication is because the aggregate data per address is multiplied by the total number of transactions the address has across all timesteps. The duplication is the result of the way the data was joined by the original owners.

In this example, '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b' has 26 total transactions across 14 timesteps, as seen in the two fields 'total_txs' and 'num_timesteps_appeared_in'. We can see that 12 transactions were sent by the address to other addresses and 14 were received by the address from other addresses.

In order to remove duplicates in 'df_wallets_features', we need to remove the 'time step' field and then remove the duplicates.


In [ ]:
df_wallets_features[df_wallets_features['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,Time step,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
17919,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,1,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
56557,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,1,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
141174,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,47,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
166680,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,44,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
192495,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,24,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000
286565,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,5,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.000

Remove the 'timestep' field and then remove duplicates. The expected result is a de-duplicated dataset with 822,942 representing 1 row per address.

In [ ]:
df_wallets_features_clean = df_wallets_features.drop(columns=['Time step'])
df_wallets_features_clean = df_wallets_features_clean.drop_duplicates()
df_wallets_features_clean.shape

(822942, 56)

Now we can see that the 'Time step' field and the duplicates have been removed. The outcome is 1 row per address, resulting in 822,942 rows and 57 columns.

In [ ]:
df_wallets_features_clean[df_wallets_features_clean['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
17919,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000


The final step in cleaning the 'df_wallets_features' dataset is to save it as a separate table in BigQuery.

We previously defined the project, so we now need to set the ID we want to save it as.

In [ ]:
table2_id = 'actor_data.wallets_features_clean'

Then we save the cleaned df as a new table.

In [ ]:
# Save DataFrame to BigQuery
to_gbq(df_wallets_features_clean, table2_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6533.18it/s]


Check that the new table we created reflects what we uploaded.

In [ ]:
# Read in combined wallet features table from BigQuery
%%bigquery df_wallets_features_clean
select * from `sixth-legend-440110-g7.actor_data.wallets_features_clean`;


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_wallets_features_clean.shape

(822942, 56)

In [ ]:
df_wallets_features_clean[df_wallets_features_clean['address'] == '1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b']

,address,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,num_timesteps_appeared_in,btc_transacted_total,btc_transacted_min,btc_transacted_max,btc_transacted_mean,btc_transacted_median,btc_sent_total,btc_sent_min,btc_sent_max,btc_sent_mean,btc_sent_median,btc_received_total,btc_received_min,btc_received_max,btc_received_mean,btc_received_median,fees_total,fees_min,fees_max,fees_mean,fees_median,fees_as_share_total,fees_as_share_min,fees_as_share_max,fees_as_share_mean,fees_as_share_median,blocks_btwn_txs_total,blocks_btwn_txs_min,blocks_btwn_txs_max,blocks_btwn_txs_mean,blocks_btwn_txs_median,blocks_btwn_input_txs_total,blocks_btwn_input_txs_min,blocks_btwn_input_txs_max,blocks_btwn_input_txs_mean,blocks_btwn_input_txs_median,blocks_btwn_output_txs_total,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
596647,1H1WS2tFx5yCsxtefs9PSTdqVX2mduyf3b,12.0000,14.0000,391200.0000,485959.0000,94759.0000,26.0000,391200.0000,399266.0000,14.0000,218.5086,0.0053,50.0000,8.4042,0.2479,66.3480,0.0000,50.0000,2.5518,0.0000,152.1606,0.0000,50.0000,5.8523,0.0125,0.0160,0.0002,0.0023,0.0006,0.0005,0.0071,0.0000,0.0012,0.0003,0.0002,94759.0000,0.0000,22178.0000,3790.3600,2011.0000,92743.0000,4.0000,24189.0000,8431.1818,6042.0000,86693.0000,0.0000,30242.0000,6668.6923,2016.0000,5.0000,62.0000,1.0000,4.0000,1.1481,1.0000


This exercise removing duplicates from the original 'wallets_features_classes_combined' and 'wallets_features' datasets is now complete.

Two additional datasets which have removed the timestamps and duplicate information per address have been created and saved in BigQuery called 'wallets_features_clean' and 'wallets_combined_clean'.